In [145]:
%%bash
pip3 install user_agents
pip3 install tqdm
pip3 install tables

In [146]:
import numpy as np
import pandas as pd
import pickle
import urllib
from tqdm import tqdm, trange, tqdm_pandas
import os
from user_agents import parse
import datetime
import time
import re
pd.set_option('display.max_columns', 1000)

---
# Bigquery to Datalab
---
(This part will not be run unless introduce new dataset)

**Load bigquery P3_Ultra_Data_P1**

In [3]:
%%time
# see if define dictionary would solve the problem
Base_Data_table = pd.read_gbq('SELECT * FROM chet_treepie.P3_Ultra_Data_1_10_S1', project_id='cloudiq-treepie-development',dialect='standard')
with open( "../Data/P3_Ultra_Data_1_10_S1.p", "wb" ) as f:
    pickle.dump(Base_Data_table, f)

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
  Got page: 2; 2% done. Elapsed 12.3 s.
  Got page: 3; 2% done. Elapsed 18.1 s.
  Got page: 4; 3% done. Elapsed 23.36 s.
  Got page: 5; 4% done. Elapsed 29.33 s.
  Got page: 6; 5% done. Elapsed 35.23 s.
  Got page: 7; 6% done. Elapsed 40.13 s.
  Got page: 8; 7% done. Elapsed 45.13 s.
  Got page: 9; 7% done. Elapsed 50.67 s.
  Got page: 10; 8% done. Elapsed 56.18 s.
  Got page: 11; 9% done. Elapsed 61.51 s.
  Got page: 12; 10% done. Elapsed 67.2 s.
  Got page: 13; 11% done. Elapsed 73.11 s.
  Got page: 14; 11% done. Elapsed 78.55 s.
  Got page: 15; 12% done. Elapsed 83.87 s.
  Got page: 16; 13% done. Elapsed 89.76 s.
  Got page: 17; 14% done. Elapsed 95.38 s.
  Got page: 18; 15% done. Elapsed 100.79 s.
  Got page: 19; 16% done. Elapsed 106.25 s.
  Got page: 20; 16% done. Elapsed 111.27 s.
  Got page: 21; 17% done. Elapsed 117.51 s.
  Got page: 22; 18% done. Elapsed 123.0 s.
  Got page: 23; 19% done. E

In [5]:
data=Base_Data_table.copy()
data.head(3)

,index,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,28145,1s6ljlakvjof314rmxutyhatcr,11b261cfde6208244452790c90ed6a30,2017-05-27 19:07:38,27,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,https%3A%2F%2Fsecure.wowcher.co.uk%2Fmyaccount...,NaT,2017-05-27 19:07:37.225,True,Rhodium-Plated White Stone Ring,92%,11.95,03:03:53:54,vivianthabo@yahoo.co.uk,None,1495911995157,None,11.95,True,None,False,2017-05-27 20:06:36,2017-05-27 20:07:35,1495912086,151.224.12.78,Mozilla%2F5.0+%28Linux%3B+Android+7.0%3B+SAMSU...,Warrington,GB
1,28145,1s6ljlakvjof314rmxutyhatcr,11b261cfde6208244452790c90ed6a30,2017-05-27 19:06:05,27,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,2017-05-27 19:06:04.546,NaT,None,Rhodium-Plated White Stone Ring,92%,11.95,03:03:53:54,vivianthabo@yahoo.co.uk,None,1495911995157,None,11.95,True,None,False,2017-05-27 20:06:36,2017-05-27 20:07:35,1495912086,151.224.12.78,Mozilla%2F5.0+%28Linux%3B+Android+7.0%3B+SAMSU...,Warrington,GB
2,28145,1s6ljlakvjof314rmxutyhatcr,11b261cfde6208244452790c90ed6a30,2017-05-27 19:07:16,27,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-27 19:07:15.459,None,Rhodium-Plated White Stone Ring,92%,11.95,03:03:53:54,vivianthabo@yahoo.co.uk,None,1495911995157,None,11.95,True,None,False,2017-05-27 20:06:36,2017-05-27 20:07:35,1495912086,151.224.12.78,Mozilla%2F5.0+%28Linux%3B+Android+7.0%3B+SAMSU...,Warrington,GB


**Get Size of Base_Data_table**

In [6]:
a=os.path.getsize('../Data/P3_Ultra_Data_1_10_S1.p')/1024/1024/1024
print('Size of P3_Ultra_Data_1_10_S1',a,'GB')

Size of P3_Ultra_Data_1_10_S1 1.1139933867380023 GB


---
# Data Initialization
---

## 1. Load data

In [10]:
%%time
with open( "../Data/P3_Ultra_Data_1_10_S1.p", "rb" ) as f:
    data=pickle.load(f)

CPU times: user 5.16 s, sys: 2.68 s, total: 7.85 s
Wall time: 1min 34s


In [11]:
print('Size of data is: ',os.path.getsize('../Data/P3_Ultra_Data_1_10_S1.p')/1024/1024/1024,'GB')
print('shape of data is: ', data.shape)

Size of data is:  1.1139933867380023 GB
shape of data is:  (1469185, 29)


In [12]:
print(data.shape)
data.head(3)

(1469185, 29)


,index,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,28145,1s6ljlakvjof314rmxutyhatcr,11b261cfde6208244452790c90ed6a30,2017-05-27 19:07:38,27,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,https%3A%2F%2Fsecure.wowcher.co.uk%2Fmyaccount...,NaT,2017-05-27 19:07:37.225,True,Rhodium-Plated White Stone Ring,92%,11.95,03:03:53:54,vivianthabo@yahoo.co.uk,None,1495911995157,None,11.95,True,None,False,2017-05-27 20:06:36,2017-05-27 20:07:35,1495912086,151.224.12.78,Mozilla%2F5.0+%28Linux%3B+Android+7.0%3B+SAMSU...,Warrington,GB
1,28145,1s6ljlakvjof314rmxutyhatcr,11b261cfde6208244452790c90ed6a30,2017-05-27 19:06:05,27,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,2017-05-27 19:06:04.546,NaT,None,Rhodium-Plated White Stone Ring,92%,11.95,03:03:53:54,vivianthabo@yahoo.co.uk,None,1495911995157,None,11.95,True,None,False,2017-05-27 20:06:36,2017-05-27 20:07:35,1495912086,151.224.12.78,Mozilla%2F5.0+%28Linux%3B+Android+7.0%3B+SAMSU...,Warrington,GB
2,28145,1s6ljlakvjof314rmxutyhatcr,11b261cfde6208244452790c90ed6a30,2017-05-27 19:07:16,27,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-27 19:07:15.459,None,Rhodium-Plated White Stone Ring,92%,11.95,03:03:53:54,vivianthabo@yahoo.co.uk,None,1495911995157,None,11.95,True,None,False,2017-05-27 20:06:36,2017-05-27 20:07:35,1495912086,151.224.12.78,Mozilla%2F5.0+%28Linux%3B+Android+7.0%3B+SAMSU...,Warrington,GB


**Rename index as id**

In [13]:
data.rename(columns={'index': 'id'}, inplace=True)

## 2. Sort data by id and datetime

In [14]:
data=data.sort_values(['id','datetime'])

**Reset Index**

In [15]:
data.reset_index(drop=True,inplace=True)

In [16]:
list = np.concatenate((range(1,5),range(-3,0)),axis=0)
print(data.shape)
data.iloc[list, :]

(1469185, 29)


,id,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
1,1,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
2,1,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
3,1,1vf4mz1acp57x1p1zlyz5xebe6,5c6eaac3c9db786d83c28c4d673ff556,2017-06-09 14:23:10,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-09 14:23:10.017,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,00:08:36:54,None,None,1497018190017,None,9,False,None,False,2017-06-09 15:23:10,2017-06-09 15:23:10,1497020106,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
4,2,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,2017-05-26 13:25:20.268,NaT,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,1495805120891,None,69,False,None,False,2017-05-26 14:25:27,2017-05-26 14:25:27,1495807024,148.252.216.66,Mozilla%2F5.0+%28Windows+NT+6.0%3B+WOW64%29+Ap...,None,GB
1469182,52433,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.554,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_2_...,Oxford,GB
1469183,52433,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.550,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_2_...,Oxford,GB
1469184,52433,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:26,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Ftravel,NaT,NaT,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_2_...,Oxford,GB


## 3. Group by id and session_id

In [17]:
df=data.groupby(['id','session_id'],sort=False).nth(0)

In [18]:
data.columns

Index(['id', 'session_id', 'finger_print', 'datetime', 'days', 'referrer',
       'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country'],
      dtype='object')

In [19]:
df=df[['finger_print', 'datetime', 'days', 'referrer',
       'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country']]

In [20]:
df['num_sess']=1
num_row=5
list = np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)
print(df.shape)
df.iloc[list, :]

(220047, 28)


finger_print  \
id    session_id                                                     
1     k79c9lk1nfop9m4wx0fmrg16    5c6eaac3c9db786d83c28c4d673ff556   
      w2gunkrgoifz0j4agqb8bz4     5c6eaac3c9db786d83c28c4d673ff556   
      14qy38xtlrcnc1lamkqq6fqd84  5c6eaac3c9db786d83c28c4d673ff556   
      1vf4mz1acp57x1p1zlyz5xebe6  5c6eaac3c9db786d83c28c4d673ff556   
2     qvsw8ty43cbb15tu52ujywjs3   e5f3ca057208f51a2cacae58ff95d757   
52432 1waht5m3hhnwa1pvz00evaj2ac  5645078a67cde02aea81f06f54e9728e   
52433 zxkbfysuold2vl2v4el3ubzv    17e96f528f8940c9392619c58b8b136d   
      vxhv5kka1iu93rtmz4g6osc4    de6f52700a2fefcd15e803abbe23cde1   
      15ox967rdf7q3yjn65eedevsh   de6f52700a2fefcd15e803abbe23cde1   
      1tz5t8tsg2bdj5c3w6i1r5ufu   3a131efaa20b6a197508313f0eb266c4   

                                            datetime  days  \
id    session_id                                             
1     k79c9lk1nfop9m4wx0fmrg16   2017-05-19 08:02:53    19   
      w2gunkrgoifz0j4agqb8bz4    2017-05-19 13:51:59    19   
      14qy38xtlrcnc1lamkqq6fqd84 2017-06-02 21:40:06     2   
      1vf4mz1acp57x1p1zlyz5xebe6 2017-06-09 14:23:10     9   
2     qvsw8ty43cbb15tu52ujywjs3  2017-05-26 13:25:27    26   
52432 1waht5m3hhnwa1pvz00evaj2ac 2017-05-20 20:26:03    20   
52433 zxkbfysuold2vl2v4el3ubzv   2017-05-24 18:48:37    24   
      vxhv5kka1iu93rtmz4g6osc4   2017-06-12 14:03:52    12   
      15ox967rdf7q3yjn65eedevsh  2017-06-12 14:03:52    12   
      1tz5t8tsg2bdj5c3w6i1r5ufu  2017-06-12 14:15:41    12   

                                                                           referrer  \
id    session_id                                                                      
1     k79c9lk1nfop9m4wx0fmrg16                                                        
      w2gunkrgoifz0j4agqb8bz4                                                         
      14qy38xtlrcnc1lamkqq6fqd84                                                      
      1vf4mz1acp57x1p1zlyz5xebe6                                                      
2     qvsw8ty43cbb15tu52ujywjs3                                                       
52432 1waht5m3hhnwa1pvz00evaj2ac          android-app%3A%2F%2Fcom.google.android.gm   
52433 zxkbfysuold2vl2v4el3ubzv    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      vxhv5kka1iu93rtmz4g6osc4    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      15ox967rdf7q3yjn65eedevsh   https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      1tz5t8tsg2bdj5c3w6i1r5ufu                                                       

                                                                           page_url  \
id    session_id                                                                      
1     k79c9lk1nfop9m4wx0fmrg16    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      w2gunkrgoifz0j4agqb8bz4     https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      14qy38xtlrcnc1lamkqq6fqd84  https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      1vf4mz1acp57x1p1zlyz5xebe6  https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
2     qvsw8ty43cbb15tu52ujywjs3   https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
52432 1waht5m3hhnwa1pvz00evaj2ac  https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
52433 zxkbfysuold2vl2v4el3ubzv    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      vxhv5kka1iu93rtmz4g6osc4    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      15ox967rdf7q3yjn65eedevsh   https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      1tz5t8tsg2bdj5c3w6i1r5ufu    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Flondon   

                                   product_viewed_browse  \
id    session_id                                           
1     k79c9lk1nfop9m4wx0fmrg16                       NaT   
      w2gunkrgoifz0j4agqb8bz4                        NaT   
      14qy38xtlrcnc1lamkqq6fqd84                     NaT   
      1vf4mz1acp57x1p1zlyz5xebe6                     NaT   
2     qvsw8ty43cbb

In [21]:
df.loc[1,['finger_print']]

,finger_print
session_id,
k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556
w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556
14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556
1vf4mz1acp57x1p1zlyz5xebe6,5c6eaac3c9db786d83c28c4d673ff556


## 4. Add num_sess column
**num_sess is timely sequential**

### 1. Add num_sess to 'df'

In [30]:
# length of the file
tmp=df.iloc[-1,:].name[0]
tmp

52433

In [31]:
# add_num_sess function for df
def add_num_sess(df):
    for i in trange(1, tmp+1):
        df.loc[i,'num_sess']=range(1, df.loc[i,['num_sess']].shape[0]+1)

In [32]:
add_num_sess(df)

100%|██████████| 52433/52433 [03:58<00:00, 219.79it/s]


In [33]:
num_row=5
df=df[['num_sess','finger_print', 'datetime', 'days', 'referrer',
       'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country']]
list = np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)
print(df.shape)
df.iloc[list, :]

(220047, 28)


num_sess                      finger_print  \
id    session_id                                                               
1     k79c9lk1nfop9m4wx0fmrg16           1  5c6eaac3c9db786d83c28c4d673ff556   
      w2gunkrgoifz0j4agqb8bz4            2  5c6eaac3c9db786d83c28c4d673ff556   
      14qy38xtlrcnc1lamkqq6fqd84         3  5c6eaac3c9db786d83c28c4d673ff556   
      1vf4mz1acp57x1p1zlyz5xebe6         4  5c6eaac3c9db786d83c28c4d673ff556   
2     qvsw8ty43cbb15tu52ujywjs3          1  e5f3ca057208f51a2cacae58ff95d757   
52432 1waht5m3hhnwa1pvz00evaj2ac         4  5645078a67cde02aea81f06f54e9728e   
52433 zxkbfysuold2vl2v4el3ubzv           1  17e96f528f8940c9392619c58b8b136d   
      vxhv5kka1iu93rtmz4g6osc4           2  de6f52700a2fefcd15e803abbe23cde1   
      15ox967rdf7q3yjn65eedevsh          3  de6f52700a2fefcd15e803abbe23cde1   
      1tz5t8tsg2bdj5c3w6i1r5ufu          4  3a131efaa20b6a197508313f0eb266c4   

                                            datetime  days  \
id    session_id                                             
1     k79c9lk1nfop9m4wx0fmrg16   2017-05-19 08:02:53    19   
      w2gunkrgoifz0j4agqb8bz4    2017-05-19 13:51:59    19   
      14qy38xtlrcnc1lamkqq6fqd84 2017-06-02 21:40:06     2   
      1vf4mz1acp57x1p1zlyz5xebe6 2017-06-09 14:23:10     9   
2     qvsw8ty43cbb15tu52ujywjs3  2017-05-26 13:25:27    26   
52432 1waht5m3hhnwa1pvz00evaj2ac 2017-05-20 20:26:03    20   
52433 zxkbfysuold2vl2v4el3ubzv   2017-05-24 18:48:37    24   
      vxhv5kka1iu93rtmz4g6osc4   2017-06-12 14:03:52    12   
      15ox967rdf7q3yjn65eedevsh  2017-06-12 14:03:52    12   
      1tz5t8tsg2bdj5c3w6i1r5ufu  2017-06-12 14:15:41    12   

                                                                           referrer  \
id    session_id                                                                      
1     k79c9lk1nfop9m4wx0fmrg16                                                        
      w2gunkrgoifz0j4agqb8bz4                                                         
      14qy38xtlrcnc1lamkqq6fqd84                                                      
      1vf4mz1acp57x1p1zlyz5xebe6                                                      
2     qvsw8ty43cbb15tu52ujywjs3                                                       
52432 1waht5m3hhnwa1pvz00evaj2ac          android-app%3A%2F%2Fcom.google.android.gm   
52433 zxkbfysuold2vl2v4el3ubzv    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      vxhv5kka1iu93rtmz4g6osc4    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      15ox967rdf7q3yjn65eedevsh   https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      1tz5t8tsg2bdj5c3w6i1r5ufu                                                       

                                                                           page_url  \
id    session_id                                                                      
1     k79c9lk1nfop9m4wx0fmrg16    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      w2gunkrgoifz0j4agqb8bz4     https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      14qy38xtlrcnc1lamkqq6fqd84  https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      1vf4mz1acp57x1p1zlyz5xebe6  https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
2     qvsw8ty43cbb15tu52ujywjs3   https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
52432 1waht5m3hhnwa1pvz00evaj2ac  https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
52433 zxkbfysuold2vl2v4el3ubzv    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      vxhv5kka1iu93rtmz4g6osc4    https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      15ox967rdf7q3yjn65eedevsh   https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      1tz5t8tsg2bdj5c3w6i1r5ufu    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Flondon   

                                   product_viewed_browse  \
id    session_id                                           
1     k79c9lk1nfop9m4wx0fmrg16                       NaT   
      w2gunkrgoifz0j4agqb8bz4                        NaT  

### 2. Store df

In [34]:
%%time
# write data
with open( "../Data/df.p", "wb" ) as f:
    pickle.dump(df, f)

CPU times: user 2.18 s, sys: 403 ms, total: 2.59 s
Wall time: 3.27 s


### 3. Add num_sess to 'data0'

In [36]:
print('reset index of df:')
df0=df.reset_index()
df0.head(6)

reset index of df:


,id,session_id,num_sess,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,1,k79c9lk1nfop9m4wx0fmrg16,1,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
1,1,w2gunkrgoifz0j4agqb8bz4,2,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
2,1,14qy38xtlrcnc1lamkqq6fqd84,3,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
3,1,1vf4mz1acp57x1p1zlyz5xebe6,4,5c6eaac3c9db786d83c28c4d673ff556,2017-06-09 14:23:10,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-09 14:23:10.017,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,00:08:36:54,None,None,1497018190017,None,9,False,None,False,2017-06-09 15:23:10,2017-06-09 15:23:10,1497020106,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
4,2,qvsw8ty43cbb15tu52ujywjs3,1,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,2017-05-26 13:25:20.268,NaT,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,1495805120891,None,69,False,None,False,2017-05-26 14:25:27,2017-05-26 14:25:27,1495807024,148.252.216.66,Mozilla%2F5.0+%28Windows+NT+6.0%3B+WOW64%29+Ap...,None,GB
5,2,xe1kopsb1o573zhid3i51eds,2,e5f3ca057208f51a2cacae58ff95d757,2017-06-09 13:20:22,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,2017-06-09 13:20:30.130,NaT,None,7nt 4* All-Inclusive Croatia & Flights - Summe...,27%,229,04:09:39:29,None,None,1497014430749,None,229,False,None,False,2017-06-09 14:20:23,2017-06-09 14:20:23,1497016328,148.252.216.66,Mozilla%2F5.0+%28Windows+NT+6.0%3B+WOW64%29+Ap...,None,GB


**Inner join of 'data' and 'df':**

In [40]:
%%time
left=data.copy()
right=df0[['id','session_id','num_sess']].copy()
data0 =pd.merge(left,right,how='inner',on=['id', 'session_id'])

CPU times: user 4.62 s, sys: 449 ms, total: 5.07 s
Wall time: 5.07 s


In [41]:
data0.columns

Index(['id', 'session_id', 'finger_print', 'datetime', 'days', 'referrer',
       'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country',
       'num_sess'],
      dtype='object')

In [42]:
# Move 'num_sess' to second column
data0=data0[['id','num_sess','session_id', 'finger_print', 'datetime', 'days', 'referrer',
       'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country']]
data0.sort_values(['id','num_sess'],inplace=True)

In [43]:
data0.reset_index(drop=True, inplace=True)

In [44]:
list=np.concatenate((range(0,3),range(data0.shape[0]-3,data0.shape[0])),axis=0)
data0.loc[list,:]

,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB
1469182,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.554,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_2_...,Oxford,GB
1469183,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.550,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_2_...,Oxford,GB
1469184,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:26,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Ftravel,NaT,NaT,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_2_...,Oxford,GB


### 4. Store data0

In [45]:
%%time
# write data
with open( "../Data/data0.p", "wb" ) as f:
    pickle.dump(data0, f)

CPU times: user 14.2 s, sys: 3 s, total: 17.2 s
Wall time: 25.3 s


# Decode User_Agent Feature

## 1. Load Data0

In [7]:
%%time
with open( "../Data/data0.p", "rb" ) as f:
    data0=pickle.load(f)

CPU times: user 4.69 s, sys: 2.26 s, total: 6.95 s
Wall time: 20.6 s


## 2. Decode user_agent

**Transform user_agent dataframe into an array:**

In [46]:
user_agent=np.asarray(data0.loc[:,'user_agent'])
len(user_agent)

1469185

- **Decode user_agent:**

**user_agent is then well decoded and was made into a list.**

In [47]:
%%time
user_agent=data0.user_agent.copy()
user_agent_refined=user_agent.apply(lambda x: urllib.parse.unquote(x))

CPU times: user 14.7 s, sys: 197 ms, total: 14.9 s
Wall time: 14.9 s


- **Result:**

In [48]:
print('Before user_agent Refine:')
print(np.asarray(data0.loc[0,'user_agent']))
print('---------------------------------------------------------------------------------------------------------------------------------------------------------')
print('After user_agent Refine:')
print(user_agent_refined[1])

Before user_agent Refine:
Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_1+like+Mac+OS+X%29+AppleWebKit%2F603.1.30+%28KHTML%2C+like+Gecko%29+Version%2F10.0+Mobile%2F14E304+Safari%2F602.1
---------------------------------------------------------------------------------------------------------------------------------------------------------
After user_agent Refine:
Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like+Mac+OS+X)+AppleWebKit/603.1.30+(KHTML,+like+Gecko)+Version/10.0+Mobile/14E304+Safari/602.1


## 3. Replace user_agent column
**Transform previous user_agent data from list to dataframe**

In [49]:
user_agent_column=user_agent_refined.copy()
user_agent_column.head()

0    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
1    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
2    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
3    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
4    Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...
Name: user_agent, dtype: object

**assign new user_agent_column value to user_agent value in original dataframe**

In [50]:
data0['user_agent']=user_agent_column.copy()
data0['user_agent'].head()

0    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
1    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
2    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
3    Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
4    Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...
Name: user_agent, dtype: object

In [51]:
print(data0.shape)

num_row=5
list = np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)
data0.iloc[list, :]

(1469185, 30)


,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
3,1,4,1vf4mz1acp57x1p1zlyz5xebe6,5c6eaac3c9db786d83c28c4d673ff556,2017-06-09 14:23:10,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-09 14:23:10.017,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,00:08:36:54,None,None,1497018190017,None,9,False,None,False,2017-06-09 15:23:10,2017-06-09 15:23:10,1497020106,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
4,2,1,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,2017-05-26 13:25:20.268,NaT,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,1495805120891,None,69,False,None,False,2017-05-26 14:25:27,2017-05-26 14:25:27,1495807024,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,GB
1469180,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:26:01,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Ftravel,NaT,NaT,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like...,Oxford,GB
1469181,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:04,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,2017-06-12 14:28:04.531,NaT,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like...,Oxford,GB
1469182,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.554,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like...,Oxford,GB
1469183,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.550,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.

## 4. Save data:
**data_final is the data such that:**
1. Full entries of all history (before groupby)
2. Sorted by datatime
3. With user_agent column decoded

In [52]:
%%time
with open( "../Data/data_final.p", "wb" )  as f:
    pickle.dump(data0, f)

CPU times: user 14.2 s, sys: 2.4 s, total: 16.6 s
Wall time: 23.3 s


## 5. Check data:

In [4]:
%%time
with open( "../Data/data_final.p", "rb" ) as f:
    data_final=pickle.load(f)

CPU times: user 4.45 s, sys: 1.92 s, total: 6.37 s
Wall time: 6.37 s


In [5]:
data_final.columns

Index(['id', 'num_sess', 'session_id', 'finger_print', 'datetime', 'days',
       'referrer', 'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country'],
      dtype='object')

In [6]:
data_final[['id', 'num_sess', 'session_id', 'finger_print', 'datetime', 'days',
       'referrer', 'page_url', 'product_viewed_browse', 'product_viewed_cart','cart_started',
       'cart_startTime','cart_firstTime', 'cart_lastTime','basket_paid_browsing', 'basket_paid_cart',
       'basketInfoTime','product_name', 'product_save', 'product_price','product_time_left',  'total',  
       'payment_type','email_click_through', 'email', 'ip_address', 'user_agent','Post_Code','city', 'country']]

,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,product_viewed_cart,cart_started,cart_startTime,cart_firstTime,cart_lastTime,basket_paid_browsing,basket_paid_cart,basketInfoTime,product_name,product_save,product_price,product_time_left,total,payment_type,email_click_through,email,ip_address,user_agent,Post_Code,city,country
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,None,2017-05-19 08:02:51.292,1495180971292,2017-05-19 09:02:53,2017-05-19 09:02:53,None,False,1495182843,None,None,None,None,None,None,True,None,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,None,GB
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,None,2017-05-19 08:02:51.292,1495180971292,2017-05-19 14:51:59,2017-05-19 14:51:59,None,False,1495203782,None,None,None,None,None,None,True,None,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,None,GB
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,None,2017-06-02 21:40:06.147,1496439606147,2017-06-02 22:40:06,2017-06-02 22:40:06,None,False,1496441524,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,9,None,False,None,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,None,GB
3,1,4,1vf4mz1acp57x1p1zlyz5xebe6,5c6eaac3c9db786d83c28c4d673ff556,2017-06-09 14:23:10,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,None,2017-06-09 14:23:10.017,1497018190017,2017-06-09 15:23:10,2017-06-09 15:23:10,None,False,1497020106,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,00:08:36:54,9,None,False,None,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,None,GB
4,2,1,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,2017-05-26 13:25:20.268,None,NaT,1495805120891,2017-05-26 14:25:27,2017-05-26 14:25:27,None,False,1495807024,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,69,None,False,None,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,None,GB
5,2,1,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,NaT,None,2017-05-26 13:25:20.891,1495805120891,2017-05-26 14:25:27,2017-05-26 14:25:27,None,False,1495807024,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,69,None,False,None,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,None,GB
6,2,2,xe1kopsb1o573zhid3i51eds,e5f3ca057208f51a2cacae58ff95d757,2017-06-09 13:20:22,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,2017-06-09 13:20:30.130,None,NaT,1497014430749,2017-06-09 14:20:23,2017-06-09 14:20:23,None,False,1497016328,7nt 4* All-Inclusive Croatia & Flights - Summe...,27%,229,04:09:39:29,229,None,False,None,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,None,GB
7,2,2,xe1kopsb1o573zhid3i51eds,e5f3ca057208f51a2cacae58ff95d757,2017-06-09 13:20:23,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,NaT,None,2017-06-09 13:20:30.749,1497014430749,2017-06-09 14:20:23,2017-06-09 14:20:23,None,False,1497016328,7nt 4* All-Inclusive Croatia & Flights - Summe...,27%,229,04:09:39:29,229,None,False,None,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,None,GB
8,2,3,184grb3zg9udt1scfxr2697n5b,e5f3ca057208f51a2cacae58ff95d757,2017-06-12 08:32:26,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,2017-06-12 08:32:43.033,None,NaT,1497256364377,2017-06-12 09:32:26,2017-06-12 09:32:26,None,False,1497258247,Modern Shaggy Rug - 10 Colours & 6 Sizes!,64%,9,03:14:27:15,9,None,False,None,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,None,GB
9,2,3,184grb3zg9udt1scfxr2697n5b,e5f3ca057208f51a

---

**Get product id from url (when cart_start is nonNull)**

In [70]:
import math, time
def delete_3_digit(x):
  return np.int(math.trunc(x/1000))

def T(x):
  return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))

In [71]:
data_final['cart_startTime']=data_final['cart_startTime'].apply(lambda x: T(delete_3_digit(int(x))) if not pd.isnull(x) else 0).copy()

In [88]:
pd.set_option('display.max_colwidth', -1)
index_carStarted_nonNull=pd.isnull(data_final['cart_started'])==False
tmp=data_final[['cart_started','page_url']][index_carStarted_nonNull]
url=tmp['page_url'].apply(lambda x: urllib.parse.unquote(x))

In [126]:
import re
def deal_num(x):
  try:
    return re.search(r'\d+', x).group()
  except AttributeError:
    return x

# re.search(r'\d+', url[5]).group()

In [127]:
tmp=url.apply(lambda x: deal_num(x))
print(len(tmp))

795708


In [129]:
def non_number(x):
  try: 
    int(x)
    return True
  except ValueError:
    return False
    

In [134]:
url

0          https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i                                                                                                               
1          https://secure.wowcher.co.uk/checkout/3838997?cloudiq=010115&ito=wowcher_cloudiq_desktop_3838997_1wc7qi6z61gz91y2f7lq4eaa0i#cloudiq_session_id=1wc7qi6z61gz91y2f7lq4eaa0i                                                                                                               
2          https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=Title&blockpos=19&deal=4096138&ito=wowcher_email_National_Personalised_v3%25202017-06-02&cauth=8a1322f0-af32-446f-8004-2ff26696be4d&st_cid=9751833&st_rid=20405273                                           
3          https://www.wowcher.co.uk/deal/shop/4096138/blackout-curtains-9-99?Linkpos=View&blockpos=1&deal=4096138&ito=wowch

In [132]:
tmp[tmp.apply(lambda x:non_number(x))==False]

36         https://www.wowcher.co.uk/deals/travel/spas-country-house/spa-breaks          
38         https://www.wowcher.co.uk/deals/travel/spas-country-house/spa-breaks          
96         https://www.wowcher.co.uk/deals/travel/spas-country-house/yorkshire           
97         https://www.wowcher.co.uk/deals/travel/spas-country-house/yorkshire           
98         https://www.wowcher.co.uk/deals/travel/spas-country-house/yorkshire           
102        https://www.wowcher.co.uk/deals/travel/spas-country-house/yorkshire           
103        https://www.wowcher.co.uk/deals/travel/spas-country-house/yorkshire           
135        https://www.wowcher.co.uk/deals/travel/uk-seaside/devon                       
137        https://www.wowcher.co.uk/deals/travel/long-haul                              
138        https://www.wowcher.co.uk/deals/travel/long-haul/thailand                     
141        https://www.wowcher.co.uk/deals/travel/long-haul/america-other                
142       

In [112]:
re.search(r'\d+', url).group()

TypeError: expected string or bytes-like object

In [106]:
url[5]

'https://www.wowcher.co.uk/email-deals/travel/4009686/69pp-weekender-breaks-3nt-4-algarve?Linkpos=Image&blockpos=9&deal=4009686&ito=wowcher_email_Travel_26052017&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9703436&st_rid=5555079'

In [74]:
data_final[['id','num_sess','session_id','product_viewed_browse','product_viewed_cart','product_name','datetime','page_url','cart_started','cart_startTime','cart_firstTime']]

,id,num_sess,session_id,product_viewed_browse,product_viewed_cart,product_name,datetime,page_url,cart_started,cart_startTime,cart_firstTime
0,1,1,k79c9lk1nfop9m4wx0fmrg16,NaT,None,None,2017-05-19 08:02:53,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,2017-05-19 08:02:51.292,2017-05-19 08:02:51,2017-05-19 09:02:53
1,1,2,w2gunkrgoifz0j4agqb8bz4,NaT,None,None,2017-05-19 13:51:59,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,2017-05-19 08:02:51.292,2017-05-19 08:02:51,2017-05-19 14:51:59
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,NaT,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,2017-06-02 21:40:06,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,2017-06-02 21:40:06.147,2017-06-02 21:40:06,2017-06-02 22:40:06
3,1,4,1vf4mz1acp57x1p1zlyz5xebe6,NaT,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,2017-06-09 14:23:10,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,2017-06-09 14:23:10.017,2017-06-09 14:23:10,2017-06-09 15:23:10
4,2,1,qvsw8ty43cbb15tu52ujywjs3,2017-05-26 13:25:20.268,None,3 5 or 7nt Algarve Break & Flights,2017-05-26 13:25:27,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,NaT,2017-05-26 13:25:20,2017-05-26 14:25:27
5,2,1,qvsw8ty43cbb15tu52ujywjs3,NaT,None,3 5 or 7nt Algarve Break & Flights,2017-05-26 13:25:27,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,2017-05-26 13:25:20.891,2017-05-26 13:25:20,2017-05-26 14:25:27
6,2,2,xe1kopsb1o573zhid3i51eds,2017-06-09 13:20:30.130,None,7nt 4* All-Inclusive Croatia & Flights - Summe...,2017-06-09 13:20:22,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,NaT,2017-06-09 13:20:30,2017-06-09 14:20:23
7,2,2,xe1kopsb1o573zhid3i51eds,NaT,None,7nt 4* All-Inclusive Croatia & Flights - Summe...,2017-06-09 13:20:23,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,2017-06-09 13:20:30.749,2017-06-09 13:20:30,2017-06-09 14:20:23
8,2,3,184grb3zg9udt1scfxr2697n5b,2017-06-12 08:32:43.033,None,Modern Shaggy Rug - 10 Colours & 6 Sizes!,2017-06-12 08:32:26,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-12 08:32:44,2017-06-12 09:32:26
9,2,3,184grb3zg9udt1scfxr2697n5b,NaT,None,Modern Shaggy Rug - 10 Colours & 6 Sizes!,2017-06-12 08:32:26,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,2017-06-12 08:32:44.377,2017-06-12 08:32:44,2017-06-12 09:32:26


---

In [55]:
data_final[['basket_paid_browsing','basket_paid_cart']][:5]
# data_final['basket_paid_browsing']

,basket_paid_browsing,basket_paid_cart
0,None,False
1,None,False
2,None,False
3,None,False
4,None,False


In [ ]:
['basket_paid_browsing','basket_paid_cart']

In [52]:
sum(data_final['basket_paid_browsing'].apply(lambda x: x!=None))

20458

In [61]:
print(sum(data_final['basket_paid_cart'].apply(lambda x: x!='False')))
print(sum(data_final['basket_paid_cart'].apply(lambda x: x=='True')))

376486
376486


---
# Decode url to product id

## Load data:

In [189]:
%%time
with open( "../Data/data_final.p", "rb" ) as f:
    data_final=pickle.load(f)

CPU times: user 4.8 s, sys: 1.25 s, total: 6.05 s
Wall time: 6.74 s


## 1. URL with product_viewed!=None 
**product_viewed_browse**

**Preprocess:**
<br>get url_product_viewed_tmp ready to URL_Parse

In [190]:
data_final['product_viewed_browse'][:10]

0   NaT                    
1   NaT                    
2   NaT                    
3   NaT                    
4   2017-05-26 13:25:20.268
5   NaT                    
6   2017-06-09 13:20:30.130
7   NaT                    
8   2017-06-12 08:32:43.033
9   NaT                    
Name: product_viewed_browse, dtype: datetime64[ns]

In [191]:
data_final['product_viewed_browse'].isnull()[:10]

0    True 
1    True 
2    True 
3    True 
4    False
5    True 
6    False
7    True 
8    False
9    True 
Name: product_viewed_browse, dtype: bool

In [192]:
tmp0=data_final['product_viewed_browse'].isnull()
# data_final such that there's a product view for that browse:
url_product_viewed_tmp=data_final[tmp0==False]['page_url']
url_product_viewed_tmp[:3]

4    https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%2Ftravel%2F4009686%2F69pp-weekender-breaks-3nt-4-algarve%3FLinkpos%3DImage%26blockpos%3D9%26deal%3D4009686%26ito%3Dwowcher_email_Travel_26052017%26cauth%3D1101ee6f-9a13-4c30-9d35-05b605b48795%26st_cid%3D9703436%26st_rid%3D5555079          
6    https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%2Ftravel%2F4085525%2F7nt-4-star-inclusive-dubrovnik-croatia-break%3FLinkpos%3DImage%26blockpos%3D10%26deal%3D4085525%26ito%3Dwowcher_email_Travel_09062017%26cauth%3D1101ee6f-9a13-4c30-9d35-05b605b48795%26st_cid%3D9805752%26st_rid%3D5555079
8    https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%2F4212459%2Fmodern-shaggy-rug-9%3FLinkpos%3DTitle%26blockpos%3D1%26deal%3D4212459%26ito%3Dwowcher_email_National_Default%2525202017-06-12%26cauth%3D1101ee6f-9a13-4c30-9d35-05b605b48795%26st_cid%3D9822925%26st_rid%3D5555079                 
Name: page_url, dtype: object

### 1. Decode URL

In [193]:
url_product_viewed=url_product_viewed_tmp.apply(lambda x: urllib.parse.unquote(x))
url_product_viewed[:3]

4    https://www.wowcher.co.uk/email-deals/travel/4009686/69pp-weekender-breaks-3nt-4-algarve?Linkpos=Image&blockpos=9&deal=4009686&ito=wowcher_email_Travel_26052017&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9703436&st_rid=5555079          
6    https://www.wowcher.co.uk/email-deals/travel/4085525/7nt-4-star-inclusive-dubrovnik-croatia-break?Linkpos=Image&blockpos=10&deal=4085525&ito=wowcher_email_Travel_09062017&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9805752&st_rid=5555079
8    https://www.wowcher.co.uk/deal/shop/4212459/modern-shaggy-rug-9?Linkpos=Title&blockpos=1&deal=4212459&ito=wowcher_email_National_Default%25202017-06-12&cauth=1101ee6f-9a13-4c30-9d35-05b605b48795&st_cid=9822925&st_rid=5555079                   
Name: page_url, dtype: object

### 2. Extract product_id from URL:

**Function to find product_id:**

In [194]:
def find_product_id(x):
    try:
        return re.search(r'\d{7}', x).group()
    except AttributeError:
        return x
# re.search(r'\d+', url[5]).group()
# re is a package

**A small test of RegExp:**

In [195]:
print(re.search(r'\d{7}', '123456789/@90123456789').group())

1234567


In [196]:
product_id=url_product_viewed.apply(lambda x:find_product_id(x))
print('length of both valid and Non-valid product id:', product_id.shape)
product_id

length of both valid and Non-valid product id: (419541,)


4          4009686                                                             
6          4085525                                                             
8          4212459                                                             
10         4247216                                                             
12         4009062                                                             
14         4085938                                                             
17         4085938                                                             
20         4095939                                                             
21         4095939                                                             
23         4095939                                                             
24         4031765                                                             
32         4042013                                                             
35         4042013                      

**Function: count number of valid product_id:**

In [197]:
def count_product_id(x):
    try:
        int(x)
        return 1
    except ValueError:
        return 0

In [198]:
print('number of product id:',sum(a.apply(lambda x: count_product_id(x))))

number of product id: 265469


**One example of not having product_id**
<br> Customer searched the product name. And on the result page, he could directly go to buy stage. Or yes, he is viewing a product page, where we didn't record, as all info has been on the result page already.

In [199]:
url_product_viewed[1469039]

'https://www.wowcher.co.uk/deal/london/3010684/leg-underarm-bikini-wax?searchresult=true'

**Substitute unvalid url to '0'**

In [200]:
def to_0(x):
  try:
    int(x)
    return x
  except ValueError:
    return 0

In [201]:
product_id_final=product_id.apply(lambda x:to_0(x))
print(product_id_final.shape)
product_id_final[:16]

(419541,)


4     4009686
6     4085525
8     4212459
10    4247216
12    4009062
14    4085938
17    4085938
20    4095939
21    4095939
23    4095939
24    4031765
32    4042013
35    4042013
37    0      
39    3701601
43    4031709
Name: page_url, dtype: object

### 3. Add product_id_ProductViewed

In [202]:
data_final['product_id_ProductViewed']=product_id_final

In [203]:
data_final.columns

Index(['id', 'num_sess', 'session_id', 'finger_print', 'datetime', 'days',
       'referrer', 'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country',
       'product_id_ProductViewed'],
      dtype='object')

**Show product_id_ProductViewed**

In [204]:
num_row=5
list=np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)

data_final[['product_viewed_browse','product_id_ProductViewed']].iloc[list,:]

,product_viewed_browse,product_id_ProductViewed
0,NaT,NaN
1,NaT,NaN
2,NaT,NaN
3,NaT,NaN
4,2017-05-26 13:25:20.268,4009686
1469180,NaT,NaN
1469181,2017-06-12 14:28:04.531,3997899
1469182,NaT,NaN
1469183,NaT,NaN
1469184,NaT,NaN


## 2. URL with cart_started!=None 
cart_started is from browse data and always use this!!!

#### 4. Check Labels

In [66]:
print(data0.shape)
data0.head(1)

(1469185, 30)


,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,1,1,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla%2F5.0+%28iPhone%3B+CPU+iPhone+OS+10_3_...,None,GB


In [73]:
count_true=sum(data0.basket_paid_cart == 'True')
print('Number of basket_paid_cart = True is: ',count_true)
print('Percentage of basket_paid_cart = True is: ',count_true/data0.shape[0])

Number of basket_paid_cart = True is:  376486
Percentage of basket_paid_cart = True is:  0.25625499852


**Sorty data0 by id, num_sess, basket_paid_cart**

In [81]:
data0.sort_values(['id','num_sess','basket_paid_cart'],inplace=True,ascending=[True, True, False])

In [ ]:
data0.groupby(['id','num_sess'])

In [25]:
basket_paid_cart_count = sum(data.basket_paid_cart=='True')
print('Number of basket_paid = True is', basket_paid_cart_count)
print('Percentage of basket_paid = ', basket_paid_cart_count/len(data.basket_paid_cart))

Number of basket_paid = True is 357436
Percentage of basket_paid =  0.22421602864


In [82]:
df_groupby2 = data0.groupby(['id','num_sess']).nth(0)

In [88]:
basket_paid_cart_count=sum(df_groupby2.basket_paid_cart == 'True')
print('Number of basket_paid = True :', basket_paid_cart_count)
print('Percentage of basket_paid : ', basket_paid_cart_count/len(df_groupby2.basket_paid_cart))

Number of basket_paid = True : 33677
Percentage of basket_paid :  0.15304457684


---
# Appendix

In [ ]:
# itertuples example for adding num_sess. 
# Not efficient at all, only 2 or 3 iterations/second
for tuple in tqdm(data0.itertuples()):
    i = tuple[0]+1
    # if id is same:
    if data0.loc[i,'id'] == data0.loc[i-1,'id']:
        # session_id is same:
        if data0.loc[i,'session_id']==data0.loc[i-1,'session_id']:  
            data0.loc[i,'num_sess']=data0.loc[i-1,'num_sess']
        # session_id is different:
        else:
            data0.loc[i,'num_sess']=data0.loc[i-1,'num_sess']+1
    else:
    # session_id is different:
      data0.loc[i,'num_sess']=1

In [16]:
data[['id','session_id']].apply(lambda x: str(x))

<bound method NDFrame.head of id            0              1\n1              1\n2         ...
session_id    0            k79c9lk1nfop9m4wx0fmrg16\n1      ...
dtype: object>

In [ ]:
# for loop example for adding num_sess.
data=data[:5000]
data['num_sess']=1

for i in trange(1,data.shape[0]):
    # if id is same:
    if data.iloc[i,0] == data.iloc[i-1,0]:   
        # session_id is same:
        if data.iloc[i,1]==data.iloc[i-1,1]:  
            data.iloc[i,-1]=data.iloc[i-1,-1]
        # session_id is different:
        else:
            data.iloc[i,-1]=data.iloc[i-1,-1]+1
    else:
        # session_id is different:
        data.iloc[i,-1]=1

In [61]:
# test=df.groupby(['id','num_sess'])['total'].nunique()

In [62]:
# test[test.values!=1]

id      num_sess
1       9           2
835     3           2
841     3           0
1257    3           2
1347    4           2
1663    4           2
1707    3           2
3128    1           0
3340    2           0
3675    3           0
3942    6           0
4398    1           0
4696    4           0
4706    2           2
4951    2           0
5008    6           2
5071    3           0
5428    1           0
5858    5           2
5979    2           2
5983    3           0
6089    1           0
6241    1           0
6807    3           2
8644    1           0
8741    2           0
10007   2           0
10074   1           2
10245   2           0
10822   2           2
                   ..
170791  1           2
181911  1           0
183803  1           2
185679  1           0
187850  1           2
189592  1           0
190489  1           2
196221  1           0
196672  1           0
198317  1           2
198982  1           0
201278  1           2
202160  1           2
202470  1      

In [68]:
sum(test.values!=1)

178

In [128]:
df0.groupby(['index','num_sess'])